In [7]:
import torch.nn as nn
import torch
import torchvision.transforms as transforms
import torchvision
import time
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models
from sklearn.metrics import confusion_matrix
import pandas as pd
import os
import pathlib

In [2]:
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for images,_ in loader:
        images_count_in_batch = images.size(0)
#         print(images.shape)
        images =images.view(images_count_in_batch,images.size(1),-1)
        mean+=images.mean(2).sum(0)
        std+=images.std(2).sum(0)
        total_images_count+=images_count_in_batch
    mean /= total_images_count
    std /= total_images_count
    return mean,std

In [24]:
def train_transform(dataset):
    train_transforms = transforms.Compose([transforms.Resize((299,299)),transforms.ToTensor()])
    train_dataset=torchvision.datasets.ImageFolder(root=dataset,transform=train_transforms)
    train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=32,shuffle=True)
    mean,std=get_mean_std(train_loader)
    train_transforms = transforms.Compose([
        transforms.Resize((299,299)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
    ])
    data_set=torchvision.datasets.ImageFolder(root=dataset,transform=train_transforms)
    return data_set

In [25]:
path = "../Datasets/dataset_20_classes/"
dataset=train_transform(path)

In [26]:
root=pathlib.Path(path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes=classes[1:]
print(classes)

['Apple_healthy', 'Apple_unhealthy', 'Cherry_healthy', 'Cherry_unhealthy', 'Grape_healthy', 'Grape_unhealthy', 'Mango_healthy', 'Mango_unhealthy', 'Peach_healthy', 'Peach_unhealthy', 'Pepperbell_healthy', 'Pepperbell_unhealthy', 'Pomegranate_healthy', 'Pomegranate_unhealthy', 'Potato_healthy', 'Potato_unhealthy', 'Strawberry_healthy', 'Strawberry_unhealthy', 'Tomato_healthy', 'Tomato_unhealthy']


In [27]:
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset)-train_size-val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size,test_size])
print(str(train_size)+" "+str(val_size)+" "+str(test_size))

13803 1971 3945


In [28]:
def set_device():
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    return device

In [29]:
device=set_device()

In [30]:
model = models.inception_v3(pretrained=False)
classifier_input = model.fc.in_features
model.fc = nn.Linear(classifier_input, 20)
model.aux_logits = False
model.to(device)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [31]:
lr = [1E-0, 1E-1, 1E-2, 1E-3, 1E-4]
Batchs=[64]
from statistics import mean
best_accuracy=0.0
best_param=""
criterion = nn.CrossEntropyLoss()
lr_accuract_training=[]
lr_accuracy_val=[]
lr_loss_train=[]
lr_loss_val=[]

In [ ]:
for n in lr:
    print("Learning Rate: %.3f "%(n))
    optimizer = torch.optim.SGD(model.parameters(), lr=n,momentum=0.9)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0.0001)
    for Batch_size in Batchs:
        print("Batch Size: %d "%(Batch_size))
        training_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=Batch_size,shuffle=True,drop_last=False,num_workers=0)
        val_loader=torch.utils.data.DataLoader(dataset=val_dataset,batch_size=Batch_size,shuffle=False,drop_last=False,num_workers=0)
        n_epochs=50
        print("Epoch size: %d "%(n_epochs))
        epoch_count = 0
        Accuracies = [] 
        valAccuracies = []
        valLoss = []
        trainLoss = []
        t1 = time.time()
        validation_accuracy = 0
        for epoch in range(n_epochs):
            print("Epoch: %d "%(epoch+1))
            model.train()
            running_loss=0.0
            running_correct=0.0
            val_loss=0.0
            total=0
            for i,data in enumerate(training_loader):
                images,labels=data
                images=images.to(device)
                labels=labels.to(device)
                total+=labels.size(0)
                optimizer.zero_grad()
                outputs=model(images)
                _,predicted=torch.max(outputs.data,1)
                loss=criterion(outputs,labels)
                loss.backward()
                optimizer.step()
                running_loss+=loss.item()
                running_correct+=(labels==predicted).sum().item()
                traning_accuracy=(running_correct/total)*100
                    
            validation_accuracy=0
            model.eval()
            with torch.no_grad():
                val_correct=0
                val_total=0
                val_loss=0
                for data in val_loader:
                    images,val_labels=data[0].to(device), data[1].to(device)
                    images = images.to(device)
                    val_labels = val_labels.to(device)
                    outputs=model(images)
                    loss_val=criterion(outputs,val_labels)
                    val_loss+= loss_val.item()
                    _,predicted=torch.max(outputs.data,1)
                    val_correct+=(predicted == val_labels).sum().item()
                    val_total+=val_labels.size(0)
                validation_accuracy = (val_correct / val_total) *100
                    
            scheduler.step()
            epoch_loss=running_loss/len(training_loader)
            val_loss=val_loss/len(val_loader)
            epoch_accuracy=100.00* running_correct/total
            Accuracies.append(epoch_accuracy)
            valAccuracies.append(validation_accuracy)
            trainLoss.append(epoch_loss)
            valLoss.append(val_loss)
                
                
            print("Training Data: Epoch Loss: %.3f, Epoch Accuracy: %.3f, Validation Loss: %.3f,Validation Accuracy: %.3f"%(epoch_loss,epoch_accuracy,val_loss,validation_accuracy))
           
        
        print("######## epoch Finished in {} seconds ###########".format(time.time()\
                                                                    -t1))
        epoch_count+=n_epochs
        torch.save(model.state_dict(), "2-Class-InceptionV3-"+str(epoch)+"_"+str(Batch_size)+"_"+str(n)+"-epoch.pt")
            
        plt.plot(range(n_epochs),Accuracies, label="Training")
        plt.plot(range(n_epochs),valAccuracies, label="Validation")
        plt.xlabel("Folds")
        plt.ylabel("Accuracies")
        plt.title("Training vs Validation vs Test Accuracies for epoches: %d, Batch Size: %d, Learning Rate: %.3f" % (n_epochs, Batch_size, n))
        plt.legend()
        plt.show()
            
        plt.plot(range(n_epochs),trainLoss, label="Training")
        plt.plot(range(n_epochs),valLoss, label="Validation")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title("Training vs Validation vs Test Loss for epoches: %d, Batch Size: %d, Learning Rate: %.3f" % (n_epochs, Batch_size, n))
        plt.legend()
        plt.show()
        
        lr_accuract_training.append(mean(Accuracies))
        lr_accuracy_val.append(mean(valAccuracies))
        lr_loss_train.append(mean(trainLoss))
        lr_loss_val.append(mean(valLoss))
            
        val=mean(valAccuracies)
        #Save the best model
        if val>best_accuracy:
            torch.save(resnet18_Model.state_dict(),'best_checkpoint.model')
            best_accuracy=val
            best_param="Learning Rate: "+str(n)+" Batch Size: "+str(Batch_size)+" "

Learning Rate: 1.000 
Batch Size: 64 
Epoch size: 50 
Epoch: 1 
Training Data: Epoch Loss: 5.492, Epoch Accuracy: 29.892, Validation Loss: 2.795,Validation Accuracy: 28.513
Epoch: 2 
Training Data: Epoch Loss: 2.676, Epoch Accuracy: 30.790, Validation Loss: 2.708,Validation Accuracy: 28.513
Epoch: 3 


In [ ]:
plt.plot(range(5),lr_accuract_training, label="Training")
plt.plot(range(5),lr_accuracy_val, label="Validation")
plt.xlabel("Folds")
plt.ylabel("Accuracies")
plt.title("Training vs Validation vs Test Accuracies with learning rate")
plt.legend()
plt.show()

In [ ]:
plt.plot(range(5),lr_loss_train, label="Training")
plt.plot(range(5),lr_loss_val, label="Validation")
plt.xlabel("Folds")
plt.ylabel("Accuracies")
plt.title("Training vs Validation vs Test Loss with learning rate")
plt.legend()
plt.show()

In [ ]:
print(best_param)

In [ ]:
model_path = 'best_checkpoint.model'

In [ ]:
state_dict = torch.load(model_path)

In [ ]:
model.load_state_dict(state_dict)

In [ ]:
testing_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
def evaluate_model(model,test_loader):
    model.eval()
    predicted_correct =0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images,labels = data
            images = images.to(device)
            labels = labels.to(device)
            total+=labels.size(0)
            outputs = model(images)
            _,predicted = torch.max(outputs,1)
            predicted_correct += (predicted == labels).sum().item()
    epoch_accuracy = 100.0* predicted_correct/total
    print("Testing Data: Epoch Accuracy: %.3f"%(epoch_accuracy))
    return epoch_accuracy

In [ ]:
evaluate_model(model,testing_loader)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay

In [ ]:
y_pred = []
y_true = []

for data in testing_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        output = resnet18_Model(inputs) 
        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) 
        labels = labels.data.cpu().numpy()
        y_true.extend(labels)

In [ ]:
confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred),display_labels=classes)
fig,ax = plt.subplots()
fig.set_size_inches(13,13)
plt.title("Confusion Matrix InceptionV3 (20-Classes)")
disp.plot(ax=ax)
plt.xticks(rotation=45)
plt.show()

In [ ]:
print("Precision Macro:{:.2f}".format(precision_score(y_true, y_pred, average='macro')))
print("Precision Micro:{:.2f}".format(precision_score(y_true,y_pred,average='micro')))
print("Recall Macro:{:.2f}".format(recall_score(y_true,y_pred,average='macro')))
print("Recall Micro:{:.2f}".format(recall_score(y_true,y_pred,average='micro')))
print("F1-Score Macro:{:.2f}".format(f1_score(y_true,y_pred,average='macro')))
print("F1-Score Micro:{:.2f}".format(f1_score(y_true,y_pred,average='micro')))